## Computer Vision con Java, TensorFlow y Jupyter - JConf 2022.


En esta charla aprenderemos como integrar Java con el famoso notebook de desarrollo de datascience, AI y ML llamado Jupyter.

**Por que Java para Machine Learning?**

Java es un lenguaje que ha ido evolucionando con el tiempo y ha ido incorporando muchas innovaciones y en particular en el area de inteligencia artificial, muestra de ello es el JSR 381 para deteccion de objetos e imagenes con Machine Learning (mas informacion [aqui](https://www.jcp.org/en/jsr/detail?id=381))

Por su parte Google tambien no se ha quedado atras y desarrollo una API de Tensorflow para Java desde los inicios de esta libreria, llamada libtensorflow: 

- Esta libreria fue perdiendo relevancia por el poco uso que se le daba debido al auge de Python en el area de DS y AI. 
- Con el auge del JSR 381 y el interes de la comunidad de Java de usarlo en ambientes de AI y ML Google comenzo con una nueva API de Tensorflow para Java, la tensorflow-core-platform.
- Google comenzo ya a descontinuar la antigua libtensorflow. Actualmente la nueva API esta en etapa experimental (alpha).

## Stack que utilizaremos.

Necesitaremos el siguiente stack:
- **Anaconda: https://www.anaconda.com.** Es una distribucion de software de DataScience, AI y Analiticas que contiene un conjunto de librerias escritas en python para desarrollar aplicaciones.
- **JDK u OpenJDK (segun su preferencia) de Java https://www.oracle.com/java/technologies/downloads/.** El Java Developer Kit para ejecutar y desarrollar aplicaciones en Java.
- **Jupyter notebook (Obtenido por el manejador de paquetes conda en Anaconda).** Este es un notebook interactivo para desarrollar y evaluar aplicaciones cientificas u otras que manejen datos. Por default utiliza el kernel de python basado en la aplicacion ipython.
- **BeakerX Jupyter kernel (Obtenido por el manejador de paquetes conda en Anaconda).** Beakerx es un jupyter kernel desarrollado de manera open source por la compania Two-Sigma la cual es una compania de tecnologia de datos e inteligencia artificial enfocada en el ambito financiero. El kernel es un kernel poliglota que soporta multiples lenguajes desde Python hasta Java, e incluso ofrece integracion con ApacheSpark para tambien desarrollar sistemas de Big Data y ML.
- **tensorflow-core-api y tensorflow-framework** (instalados via Maven en Jupyter, ver mas abajo). La libreria de Machine Learning y computo cientifico de Google.

### Instalacion de Anaconda y JDK.

El proceso de instalacion es sencillo en este caso, solo basta con descargar el ejecutable de ambas aplicaciones, hacer doble clic en el y seguir los pasos de instalacion. Esto aplica para todas las plataformas (Linux, macOS y Windows).

### Instalacion de Jupyter y BeakerX.

Creamos un conda environment llamado beakerx:

`$ conda create -y -n beakerx 'python>=3'`

Activamos el environment de beakerx de la siguiente manera:

`$ conda activate beakerx`

Notara que el prompt de su terminal cambiara a `(beakerx)$` indicando que se ha cambiado correctamente al environment de beakerx.

Configuramos el environment para que apunte al OpenJDK o JDK que usted tiene en su maquina (en mi caso tengo el JDK v18);

`(beakerx)$ conda config --env --add pinned_packages 'jdk>8.0.121'`.

Instalamos Jupyter y Beakerx:

`(beakerx)$ conda install -y -c conda-forge ipywidgets beakerx`

Para correr jupyter con beakerx:

`(beakerx)$ jupyter notebook`


### Instalando Google Tensorflow desde Maven

Utilizamos el jupyter magic %classpath para instalar y precompilar a traves de maven la libreria de Google Tensorflow Java API.

In [1]:
%classpath add mvn org.tensorflow:tensorflow-core-platform:0.4.1
%classpath add mvn org.tensorflow:tensorflow-framework:0.4.1

### Creando una clase on los metodos para manejar el dataset de imagenes.

El dataset que estaremos usando aca es el dataset del MNIST (mas info aqui: http://yann.lecun.com/exdb/mnist/) que tiene imagenes de digitos del 0 al 9 escritos a mano. El modelo se entrenara de manera que pueda reconocer una imagen de un digito manuscrito y decir que digito es ese.

Con esta clase podremos tener todos los metodos/funciones que necesitamos para manejar el dataset de imagenes y tambien podemos generar la lista de archivos de entrenamiento (training set), archivos de prueba (test set).


![MNIST](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

In [2]:
import static org.tensorflow.ndarray.index.Indices.range;
import java.util.Iterator;
import org.tensorflow.ndarray.index.Index;
import org.tensorflow.ndarray.ByteNdArray;
import org.tensorflow.ndarray.index.Index;
import org.tensorflow.ndarray.Shape;
import org.tensorflow.ndarray.buffer.DataBuffers;
import org.tensorflow.ndarray.ByteNdArray;
import org.tensorflow.ndarray.NdArrays;
import java.io.DataInputStream;
import java.io.IOException;
import java.util.zip.GZIPInputStream;
import static org.tensorflow.ndarray.index.Indices.sliceFrom;
import static org.tensorflow.ndarray.index.Indices.sliceTo;

/** Basic batch iterator across images presented in datset. */
class ImageBatchIterator implements Iterator<ImageBatch> {

  @Override
  public boolean hasNext() {
    return batchStart < numImages;
  }

  @Override
  public ImageBatch next() {
    long nextBatchSize = Math.min(batchSize, numImages - batchStart);
    Index range = range(batchStart, batchStart + nextBatchSize);
    batchStart += nextBatchSize;
    return new ImageBatch(images.slice(range), labels.slice(range));
  }

  public ImageBatchIterator(int batchSize, ByteNdArray images, ByteNdArray labels) {
    this.batchSize = batchSize;
    this.images = images;
    this.labels = labels;
    this.numImages = images != null ? images.shape().size(0) : 0;
    this.batchStart = 0;
  }

  private final int batchSize;
  private final ByteNdArray images;
  private final ByteNdArray labels;
  private final long numImages;
  private int batchStart;
}

class ImageBatch {
  
  public ByteNdArray images() {
    return images;
  }
  
  public ByteNdArray labels() {
    return labels;
  }

  public ImageBatch(ByteNdArray images, ByteNdArray labels) {
    this.images = images;
    this.labels = labels;
  }

  private final ByteNdArray images;
  private final ByteNdArray labels;
}


class MnistDataset {
  public static final int NUM_CLASSES = 10;

  public static MnistDataset create(int validationSize, String trainingImagesArchive, String trainingLabelsArchive,
                                    String testImagesArchive, String testLabelsArchive) {
    try {
      System.out.println(trainingImagesArchive);
      ByteNdArray trainingImages = readArchive(trainingImagesArchive);
      ByteNdArray trainingLabels = readArchive(trainingLabelsArchive);
      ByteNdArray testImages = readArchive(testImagesArchive);
      ByteNdArray testLabels = readArchive(testLabelsArchive);
      ByteNdArray empty;

      if (validationSize > 0) {
        return new MnistDataset(
            trainingImages.slice(sliceFrom(validationSize)),
            trainingLabels.slice(sliceFrom(validationSize)),
            trainingImages.slice(sliceTo(validationSize)),
            trainingLabels.slice(sliceTo(validationSize)),
            testImages,
            testLabels
        );
      }
      
      return new MnistDataset(trainingImages, trainingLabels, null, null, testImages, testLabels);
      

    } catch (IOException e) {
        throw new AssertionError(e);
    }
  }

  public Iterable<ImageBatch> trainingBatches(int batchSize) {
    return () -> new ImageBatchIterator(batchSize, trainingImages, trainingLabels);
  }

  public Iterable<ImageBatch> validationBatches(int batchSize) {
    return () -> new ImageBatchIterator(batchSize, validationImages, validationLabels);
  }

  public Iterable<ImageBatch> testBatches(int batchSize) {
    return () -> new ImageBatchIterator(batchSize, testImages, testLabels);
  }

  public ImageBatch testBatch() {
    return new ImageBatch(testImages, testLabels);
  }

  public long imageSize() {
    return imageSize;
  }

  public long numTrainingExamples() {
    return trainingLabels.shape().size(0);
  }

  public long numTestingExamples() {
    return testLabels.shape().size(0);
  }

  public long numValidationExamples() {
    return validationLabels.shape().size(0);
  }

  private static final int TYPE_UBYTE = 0x08;

  private final ByteNdArray trainingImages;
  private final ByteNdArray trainingLabels;
  private final ByteNdArray validationImages;
  private final ByteNdArray validationLabels;
  private final ByteNdArray testImages;
  private final ByteNdArray testLabels;
  private final long imageSize;

  private MnistDataset(
      ByteNdArray trainingImages,
      ByteNdArray trainingLabels,
      ByteNdArray validationImages,
      ByteNdArray validationLabels,
      ByteNdArray testImages,
      ByteNdArray testLabels
  ) {
    this.trainingImages = trainingImages;
    this.trainingLabels = trainingLabels;
    this.validationImages = validationImages;
    this.validationLabels = validationLabels;
    this.testImages = testImages;
    this.testLabels = testLabels;
    this.imageSize = trainingImages.get(0).shape().size();
  }

  private static ByteNdArray readArchive(String archiveName) throws IOException {
    DataInputStream archiveStream = new DataInputStream(
        //new GZIPInputStream(new java.io.FileInputStream("src/main/resources/"+archiveName))
        new GZIPInputStream(new java.io.FileInputStream(archiveName))
    );
    archiveStream.readShort(); // first two bytes are always 0
    byte magic = archiveStream.readByte();
    if (magic != TYPE_UBYTE) {
      throw new IllegalArgumentException("\"" + archiveName + "\" is not a valid archive");
    }
    int numDims = archiveStream.readByte();
    long[] dimSizes = new long[numDims];
    int size = 1;  // for simplicity, we assume that total size does not exceeds Integer.MAX_VALUE
    
    for (int i = 0; i < dimSizes.length; ++i) {
      dimSizes[i] = archiveStream.readInt();
      size *= dimSizes[i];
    }
    
    byte[] bytes = new byte[size];
    archiveStream.readFully(bytes);
    return NdArrays.wrap(Shape.of(dimSizes), DataBuffers.of(bytes, true, false));
  }
}

com.twosigma.beaker.javash.bkr572f4c5e.ImageBatchIterator

### Red Neuronal Convolucional (CNN).

Una red neuronal convolucional es un tipo de red neuronal muy usado en Inteligencia Artificial para variadas aplicaciones, entre ellas la mas popular la clasificiacion o reconocimiento de imagenes. Utiliza el mecanismo de deep learning (aprendizaje profundo) porque la red neuronal se compone de varias capas que secuencialmente van pasando la salida de una hacia la entrada de la otra, donde cada capa se enfoca en extraer algun tipo de feature o caracteristica del dato que se esta aprendiendo. El tipo de aprendizaje que estamos aplicando es aprendizaje supervisado (Supervised Learning) porque le estamos dando los datos y las etiquetas de cada uno para que la maquina aprenda a identificarlos.

![CNN](https://editor.analyticsvidhya.com/uploads/59954intro%20to%20CNN.JPG)
![Filters](https://www.baeldung.com/wp-content/uploads/sites/4/2022/03/conv.png)
![OneHot encoding](https://miro.medium.com/max/1400/1*ggtP4a5YaRx6l09KQaYOnw.png)

In [3]:
import java.util.Arrays;
import java.util.logging.Level;
import java.util.logging.Logger;
import org.tensorflow.Graph;
import org.tensorflow.Operand;
import org.tensorflow.Session;
import org.tensorflow.framework.optimizers.AdaDelta;
import org.tensorflow.framework.optimizers.AdaGrad;
import org.tensorflow.framework.optimizers.AdaGradDA;
import org.tensorflow.framework.optimizers.Adam;
import org.tensorflow.framework.optimizers.GradientDescent;
import org.tensorflow.framework.optimizers.Momentum;
import org.tensorflow.framework.optimizers.Optimizer;
import org.tensorflow.framework.optimizers.RMSProp;
import org.tensorflow.ndarray.ByteNdArray;
import org.tensorflow.ndarray.FloatNdArray;
import org.tensorflow.ndarray.Shape;
import org.tensorflow.ndarray.index.Indices;
import org.tensorflow.op.Op;
import org.tensorflow.op.Ops;
import org.tensorflow.op.core.Constant;
import org.tensorflow.op.core.OneHot;
import org.tensorflow.op.core.Placeholder;
import org.tensorflow.op.core.Reshape;
import org.tensorflow.op.core.Variable;
import org.tensorflow.op.math.Add;
import org.tensorflow.op.math.Mean;
import org.tensorflow.op.nn.Conv2d;
import org.tensorflow.op.nn.MaxPool;
import org.tensorflow.op.nn.Relu;
import org.tensorflow.op.nn.Softmax;
import org.tensorflow.op.nn.SoftmaxCrossEntropyWithLogits;
import org.tensorflow.op.random.TruncatedNormal;
import org.tensorflow.types.TFloat32;
import org.tensorflow.types.TUint8;


public class CnnMnist {

  private static final Logger logger = Logger.getLogger(CnnMnist.class.getName());

  private static final int PIXEL_DEPTH = 255;
  private static final int NUM_CHANNELS = 1;
  private static final int IMAGE_SIZE = 28;
  private static final int NUM_LABELS = MnistDataset.NUM_CLASSES;
  private static final long SEED = 123456789L;

  private static final String PADDING_TYPE = "SAME";

  public static final String INPUT_NAME = "input";
  public static final String OUTPUT_NAME = "output";
  public static final String TARGET = "target";
  public static final String TRAIN = "train";
  public static final String TRAINING_LOSS = "training_loss";

  public static Graph build(String optimizerName) {
    Graph graph = new Graph();

    Ops tf = Ops.create(graph);

    // Inputs
    Placeholder<TUint8> input = tf.withName(INPUT_NAME).placeholder(TUint8.class,
        Placeholder.shape(Shape.of(-1, IMAGE_SIZE, IMAGE_SIZE)));
    Reshape<TUint8> input_reshaped = tf
        .reshape(input, tf.array(-1, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS));
    Placeholder<TUint8> labels = tf.withName(TARGET).placeholder(TUint8.class);

    // Scaling the features we must make sure that images are properly centered and scaled to the same size.
    Constant<TFloat32> centeringFactor = tf.constant(PIXEL_DEPTH / 2.0f);
    Constant<TFloat32> scalingFactor = tf.constant((float) PIXEL_DEPTH);
    Operand<TFloat32> scaledInput = tf.math
        .div(tf.math.sub(tf.dtypes.cast(input_reshaped, TFloat32.class), centeringFactor),
            scalingFactor);

    // First conv layer
    Variable<TFloat32> conv1Weights = tf.variable(tf.math.mul(tf.random
        .truncatedNormal(tf.array(5, 5, NUM_CHANNELS, 32), TFloat32.class,
            TruncatedNormal.seed(SEED)), tf.constant(0.1f)));
    Conv2d<TFloat32> conv1 = tf.nn
        .conv2d(scaledInput, conv1Weights, Arrays.asList(1L, 1L, 1L, 1L), PADDING_TYPE);
    Variable<TFloat32> conv1Biases = tf
        .variable(tf.fill(tf.array(new int[]{32}), tf.constant(0.0f)));
    Relu<TFloat32> relu1 = tf.nn.relu(tf.nn.biasAdd(conv1, conv1Biases));

    // First pooling layer
    MaxPool<TFloat32> pool1 = tf.nn
        .maxPool(relu1, tf.array(1, 2, 2, 1), tf.array(1, 2, 2, 1),
            PADDING_TYPE);

    // Second conv layer
    Variable<TFloat32> conv2Weights = tf.variable(tf.math.mul(tf.random
        .truncatedNormal(tf.array(5, 5, 32, 64), TFloat32.class,
            TruncatedNormal.seed(SEED)), tf.constant(0.1f)));
    Conv2d<TFloat32> conv2 = tf.nn
        .conv2d(pool1, conv2Weights, Arrays.asList(1L, 1L, 1L, 1L), PADDING_TYPE);
    Variable<TFloat32> conv2Biases = tf
        .variable(tf.fill(tf.array(new int[]{64}), tf.constant(0.1f)));
    Relu<TFloat32> relu2 = tf.nn.relu(tf.nn.biasAdd(conv2, conv2Biases));

    // Second pooling layer
    MaxPool<TFloat32> pool2 = tf.nn
        .maxPool(relu2, tf.array(1, 2, 2, 1), tf.array(1, 2, 2, 1),
            PADDING_TYPE);

    // Flatten inputs
    Reshape<TFloat32> flatten = tf.reshape(pool2, tf.concat(Arrays
        .asList(tf.slice(tf.shape(pool2), tf.array(new int[]{0}), tf.array(new int[]{1})),
            tf.array(new int[]{-1})), tf.constant(0)));

    // Fully connected layer
    Variable<TFloat32> fc1Weights = tf.variable(tf.math.mul(tf.random
        .truncatedNormal(tf.array(IMAGE_SIZE * IMAGE_SIZE * 4, 512), TFloat32.class,
            TruncatedNormal.seed(SEED)), tf.constant(0.1f)));
    Variable<TFloat32> fc1Biases = tf
        .variable(tf.fill(tf.array(new int[]{512}), tf.constant(0.1f)));
    Relu<TFloat32> relu3 = tf.nn
        .relu(tf.math.add(tf.linalg.matMul(flatten, fc1Weights), fc1Biases));

    // Softmax layer
    Variable<TFloat32> fc2Weights = tf.variable(tf.math.mul(tf.random
        .truncatedNormal(tf.array(512, NUM_LABELS), TFloat32.class,
            TruncatedNormal.seed(SEED)), tf.constant(0.1f)));
    Variable<TFloat32> fc2Biases = tf
        .variable(tf.fill(tf.array(new int[]{NUM_LABELS}), tf.constant(0.1f)));

    Add<TFloat32> logits = tf.math.add(tf.linalg.matMul(relu3, fc2Weights), fc2Biases);

    // Predicted outputs
    Softmax<TFloat32> prediction = tf.withName(OUTPUT_NAME).nn.softmax(logits);

    // Loss function & regularization
    OneHot<TFloat32> oneHot = tf
        .oneHot(labels, tf.constant(10), tf.constant(1.0f), tf.constant(0.0f));
    SoftmaxCrossEntropyWithLogits<TFloat32> batchLoss = tf.nn.softmaxCrossEntropyWithLogits(logits, oneHot);
    Mean<TFloat32> labelLoss = tf.math.mean(batchLoss.loss(), tf.constant(0));
    Add<TFloat32> regularizers = tf.math.add(tf.nn.l2Loss(fc1Weights), tf.math
        .add(tf.nn.l2Loss(fc1Biases),
            tf.math.add(tf.nn.l2Loss(fc2Weights), tf.nn.l2Loss(fc2Biases))));
    Add<TFloat32> loss = tf.withName(TRAINING_LOSS).math
        .add(labelLoss, tf.math.mul(regularizers, tf.constant(5e-4f)));

    String lcOptimizerName = optimizerName.toLowerCase();
    // Optimizer
    Optimizer optimizer;
    switch (lcOptimizerName) {
      case "adadelta":
        optimizer = new AdaDelta(graph, 1f, 0.95f, 1e-8f);
        break;
      case "adagradda":
        optimizer = new AdaGradDA(graph, 0.01f);
        break;
      case "adagrad":
        optimizer = new AdaGrad(graph, 0.01f);
        break;
      case "adam":
        optimizer = new Adam(graph, 0.001f, 0.9f, 0.999f, 1e-8f);
        break;
      case "sgd":
        optimizer = new GradientDescent(graph, 0.01f);
        break;
      case "momentum":
        optimizer = new Momentum(graph, 0.01f, 0.9f, false);
        break;
      case "rmsprop":
        optimizer = new RMSProp(graph, 0.01f, 0.9f, 0.0f, 1e-10f, false);
        break;
      default:
        throw new IllegalArgumentException("Unknown optimizer " + optimizerName);
    }
    logger.info("Optimizer = " + optimizer);
    Op minimize = optimizer.minimize(loss, TRAIN);

    return graph;
  }

  public static void train(Session session, int epochs, int minibatchSize, MnistDataset dataset) {
    int interval = 0;
    // Train the model
    for (int i = 0; i < epochs; i++) {
      for (ImageBatch trainingBatch : dataset.trainingBatches(minibatchSize)) {
        try (TUint8 batchImages = TUint8.tensorOf(trainingBatch.images());
            TUint8 batchLabels = TUint8.tensorOf(trainingBatch.labels());
            TFloat32 loss = (TFloat32)session.runner()
                .feed(TARGET, batchLabels)
                .feed(INPUT_NAME, batchImages)
                .addTarget(TRAIN)
                .fetch(TRAINING_LOSS)
                .run().get(0)) {
          if (interval % 100 == 0) {
            logger.log(Level.INFO,
                "Iteration = " + interval + ", training loss = " + loss.getFloat());
          }
        }
        interval++;
      }
    }
  }

  public static void test(Session session, int minibatchSize, MnistDataset dataset) {
    int correctCount = 0;
    int[][] confusionMatrix = new int[10][10];

    for (ImageBatch trainingBatch : dataset.testBatches(minibatchSize)) {
      try (TUint8 transformedInput = TUint8.tensorOf(trainingBatch.images());
          TFloat32 outputTensor = (TFloat32)session.runner()
              .feed(INPUT_NAME, transformedInput)
              .fetch(OUTPUT_NAME).run().get(0)) {

        ByteNdArray labelBatch = trainingBatch.labels();
        for (int k = 0; k < labelBatch.shape().size(0); k++) {
          byte trueLabel = labelBatch.getByte(k);
          int predLabel;

          predLabel = argmax(outputTensor.slice(Indices.at(k), Indices.all()));
          if (predLabel == trueLabel) {
            correctCount++;
          }

          confusionMatrix[trueLabel][predLabel]++;
        }
      }
    }

    logger.info("Final accuracy = " + ((float) correctCount) / dataset.numTestingExamples());

    StringBuilder sb = new StringBuilder();
    sb.append("Label");
    for (int i = 0; i < confusionMatrix.length; i++) {
      sb.append(String.format("%1$5s", "" + i));
    }
    sb.append("\n");

    for (int i = 0; i < confusionMatrix.length; i++) {
      sb.append(String.format("%1$5s", "" + i));
      for (int j = 0; j < confusionMatrix[i].length; j++) {
        sb.append(String.format("%1$5s", "" + confusionMatrix[i][j]));
      }
      sb.append("\n");
    }

    System.out.println(sb);
  }

  
  public static int argmax(FloatNdArray probabilities) {
    float maxVal = Float.NEGATIVE_INFINITY;
    int idx = 0;
    for (int i = 0; i < probabilities.shape().size(0); i++) {
      float curVal = probabilities.getFloat(i);
      if (curVal > maxVal) {
        maxVal = curVal;
        idx = i;
      }
    }
    return idx;
  }
}

com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist

### Entrenando la red neuronal

En los siguientes pasos entrenaremos la red neuronal utilizando 10 epochs o iteraciones.

In [ ]:
import java.util.logging.Logger;
import org.tensorflow.Graph;
import org.tensorflow.Session;
import java.nio.file.Paths;
import java.lang.*;

int epochs = 10;
int minibatchSize = 10;

String currentDirectory = Paths.get("").toAbsolutePath().toString() + "/";
System.out.println("Current directory: " + currentDirectory);

final String TRAINING_IMAGES_ARCHIVE = currentDirectory + "mnist-dataset/train-images-idx3-ubyte.gz";
final String TRAINING_LABELS_ARCHIVE = currentDirectory + "mnist-dataset/train-labels-idx1-ubyte.gz";
final String TEST_IMAGES_ARCHIVE = currentDirectory + "mnist-dataset/t10k-images-idx3-ubyte.gz";
final String TEST_LABELS_ARCHIVE = currentDirectory + "mnist-dataset/t10k-labels-idx1-ubyte.gz";

Logger logger = Logger.getLogger("Main logger");
MnistDataset dataset = MnistDataset.create(0, TRAINING_IMAGES_ARCHIVE, TRAINING_LABELS_ARCHIVE, TEST_IMAGES_ARCHIVE, TEST_LABELS_ARCHIVE);

logger.info("Loaded data.");

try (Graph graph = CnnMnist.build("RMSProp");    
  Session session = new Session(graph)) {
  CnnMnist.train(session, epochs, minibatchSize, dataset);

  logger.info("Trained model");

  CnnMnist.test(session, minibatchSize, dataset);
} catch (Exception e) {
    System.out.println(e);
}

Current directory: /Users/juanjpolanco/Documents/CV_with_Java/cv-tensorflow-jupyter-java-jconf2022/
/Users/juanjpolanco/Documents/CV_with_Java/cv-tensorflow-jupyter-java-jconf2022/mnist-dataset/train-images-idx3-ubyte.gz


Aug 03, 2022 10:07:32 PM com.twosigma.beaker.javash.bkr572f4c5e.BeakerWrapperClass1261714175Id967e7e312e6f4edb8e5ba46c66cd30d6 beakerRun
INFO: Loaded data.
Aug 03, 2022 10:07:35 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist build
INFO: Optimizer = RMSProp{learningRate=0.01, decay=0.9, momentum=0.0, epsilon=1.0E-10, centered=false}
Aug 03, 2022 10:07:35 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 0, training loss = 6.570856
Aug 03, 2022 10:07:37 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 100, training loss = 4.7705083
Aug 03, 2022 10:07:39 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 200, training loss = 0.75649226
Aug 03, 2022 10:07:40 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 300, training loss = 0.40414923
Aug 03, 2022 10:07:42 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 400, training loss = 0.31215405
Aug 03, 2022 10:07:44 

Aug 03, 2022 10:09:13 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 6100, training loss = 0.05490373
Aug 03, 2022 10:09:14 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 6200, training loss = 0.41990888
Aug 03, 2022 10:09:16 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 6300, training loss = 0.16968188
Aug 03, 2022 10:09:18 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 6400, training loss = 0.03487949
Aug 03, 2022 10:09:19 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 6500, training loss = 0.037381288
Aug 03, 2022 10:09:21 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 6600, training loss = 0.03453203
Aug 03, 2022 10:09:22 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 6700, training loss = 0.2577493
Aug 03, 2022 10:09:24 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Itera

Aug 03, 2022 10:10:52 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 12400, training loss = 0.04647497
Aug 03, 2022 10:10:53 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 12500, training loss = 0.5341081
Aug 03, 2022 10:10:55 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 12600, training loss = 0.032085117
Aug 03, 2022 10:10:56 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 12700, training loss = 0.21753564
Aug 03, 2022 10:10:58 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 12800, training loss = 0.18725315
Aug 03, 2022 10:10:59 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 12900, training loss = 0.05784621
Aug 03, 2022 10:11:01 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 13000, training loss = 0.03305897
Aug 03, 2022 10:11:02 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO

Aug 03, 2022 10:12:32 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 18800, training loss = 0.55106115
Aug 03, 2022 10:12:33 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 18900, training loss = 0.030496743
Aug 03, 2022 10:12:35 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 19000, training loss = 0.03049314
Aug 03, 2022 10:12:37 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 19100, training loss = 0.039093934
Aug 03, 2022 10:12:38 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 19200, training loss = 0.04146476
Aug 03, 2022 10:12:40 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 19300, training loss = 0.11316521
Aug 03, 2022 10:12:41 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 19400, training loss = 0.5713376
Aug 03, 2022 10:12:43 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INF

...

Aug 03, 2022 10:19:19 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 44000, training loss = 0.017995887
Aug 03, 2022 10:19:20 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 44100, training loss = 0.02008864
Aug 03, 2022 10:19:22 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 44200, training loss = 0.017961688
Aug 03, 2022 10:19:23 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 44300, training loss = 0.018459242
Aug 03, 2022 10:19:25 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 44400, training loss = 0.5893866
Aug 03, 2022 10:19:27 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 44500, training loss = 0.018900836
Aug 03, 2022 10:19:28 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 44600, training loss = 0.018247832
Aug 03, 2022 10:19:30 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train


Aug 03, 2022 10:21:00 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 50300, training loss = 0.10993437
Aug 03, 2022 10:21:02 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 50400, training loss = 0.02694911
Aug 03, 2022 10:21:03 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 50500, training loss = 0.020752639
Aug 03, 2022 10:21:05 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 50600, training loss = 0.01825488
Aug 03, 2022 10:21:07 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 50700, training loss = 0.023670116
Aug 03, 2022 10:21:08 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 50800, training loss = 0.017575016
Aug 03, 2022 10:21:10 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 50900, training loss = 0.03584523
Aug 03, 2022 10:21:11 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
I

Aug 03, 2022 10:22:42 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 56600, training loss = 0.019667719
Aug 03, 2022 10:22:44 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 56700, training loss = 0.020502236
Aug 03, 2022 10:22:45 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 56800, training loss = 0.019706992
Aug 03, 2022 10:22:47 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 56900, training loss = 0.021493241
Aug 03, 2022 10:22:48 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 57000, training loss = 0.021584123
Aug 03, 2022 10:22:50 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 57100, training loss = 0.020970136
Aug 03, 2022 10:22:52 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist train
INFO: Iteration = 57200, training loss = 0.033715554
Aug 03, 2022 10:22:53 PM com.twosigma.beaker.javash.bkr572f4c5e.CnnMnist tra

Label    0    1    2    3    4    5    6    7    8    9
    0  952    0    1    0    0    0   25    0    1    1
    1    0 1099   11    1    7    1    9    0    7    0
    2   18    4  980    5    1    0   13    3    6    2
    3    0    1    3  980    0   11    6    5    1    3
    4    2    2    1    0  952    0   10    0    0   15
    5    3    2    0    9    1  858    8    1    2    8
    6    4    2    0    0    1    5  945    0    1    0
    7    0   21   23    9    4    0    4  946    3   18
    8    2    0    4   28   10    1   14    1  904   10
    9    5    2    2    4    4    0    2    3    4  983



null